In [1]:
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract opencv-python


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,698 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [2]:
!tesseract --version


tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


In [3]:
import pytesseract
print(pytesseract.get_tesseract_version())


4.1.1


In [4]:
from google.colab import files
uploaded = files.upload()


Saving sample_invoice.png to sample_invoice.png


✅ Image loaded successfully!
🔍 Extracted Text:
 Invoice #12345
Date: 15/02/2024
Total Amount: $250.75



In [8]:
import cv2
import pytesseract
import os
import re
import csv
import sqlite3

# Automatically get the uploaded filename
image_path = list(uploaded.keys())[0]

# Load the image
image = cv2.imread(image_path)

# Check if image is loaded correctly
if image is None:
    print("⚠️ Error: Image not found. Check filename and re-upload.")
else:
    print("✅ Image loaded successfully!")

    # Convert to grayscale (improves OCR accuracy)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding for better OCR results
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Extract text using Tesseract OCR
    extracted_text = pytesseract.image_to_string(gray)

    # Print the extracted text
    print("\n🔍 Extracted Text:\n", extracted_text)

    # Function to extract invoice details using regex
    def extract_invoice_details(text):
        invoice_number = re.search(r"Invoice\s*#?\s*(\d+)", text)
        date = re.search(r"(\d{2}/\d{2}/\d{4})", text)
        amount = re.search(r"\$\s?(\d+\.\d{2})", text)

        return {
            "Invoice Number": invoice_number.group(1) if invoice_number else None,
            "Date": date.group(1) if date else None,
            "Amount": amount.group(1) if amount else None,
        }

    # Extract structured data
    invoice_data = extract_invoice_details(extracted_text)
    print("\n📜 Extracted Invoice Data:", invoice_data)

    # Save extracted data to CSV file
    csv_filename = "invoices.csv"
    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([invoice_data["Invoice Number"], invoice_data["Date"], invoice_data["Amount"]])

    print(f"✅ Data saved to CSV: {csv_filename}")

    # Create or connect to SQLite database
    conn = sqlite3.connect("invoices.db")
    cursor = conn.cursor()

    # Create a table for storing invoice details
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS invoices (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        invoice_number TEXT,
        date TEXT,
        amount REAL
    )
    """)

    # Save data to SQLite database
    cursor.execute("INSERT INTO invoices (invoice_number, date, amount) VALUES (?, ?, ?)",
                   (invoice_data["Invoice Number"], invoice_data["Date"], invoice_data["Amount"]))

    conn.commit()
    conn.close()
    print("✅ Data saved to SQLite database!")

    # Query and display saved invoices
    conn = sqlite3.connect("invoices.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM invoices")
    rows = cursor.fetchall()
    conn.close()

    print("\n📊 Stored Invoices in Database:")
    for row in rows:
        print(row)


✅ Image loaded successfully!

🔍 Extracted Text:
 Invoice #12345
Date: 15/02/2024
Total Amount: $250.75


📜 Extracted Invoice Data: {'Invoice Number': '12345', 'Date': '15/02/2024', 'Amount': '250.75'}
✅ Data saved to CSV: invoices.csv
✅ Data saved to SQLite database!

📊 Stored Invoices in Database:
(1, '12345', '15/02/2024', 250.75)
(2, '12345', '15/02/2024', 250.75)
